## Imports

In [ ]:
import pandas as pd
import numpy as np
from keras.layers import Activation, Dropout, Dense, Input
from tensorflow.python.keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras_preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)
df_train = pd.read_csv('gdrive/MyDrive/train_df.csv')
df_test = pd.read_csv('gdrive/MyDrive/test_df.csv')

print('Train shape: ', df_train.shape)
print('Test shape: ', df_test.shape)


Mounted at /content/gdrive/
Train shape:  (1000000, 3)
Test shape:  (306122, 2)


## TF-IDF

In [ ]:
import re, string
lightpre = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def LightPreProcessing(s): 
  return lightpre.sub(r' \1 ', s).split()
def Preprocessing(df):
    # Case correction : 
    df['question_text'] = df['question_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    # Getting rid of punctuations : 
    # df['question_text'] = df['question_text'].str.replace('[^\w\s]','')
    # # Getting rid of numbers :
    # df['question_text'] = df['question_text'].str.replace('\d','')
    # Removing Stopwords : aka useless preps and articles. Emphasis on verbs nouns and adjectives.
    # df['question_text'] = df['question_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))
    #lemmi
    from textblob import Word
    # df['question_text'] = df['question_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 
    
    lancaster = nltk.LancasterStemmer()
    df['question_text'] = df['question_text'].apply(lambda x: " ".join(lancaster.stem(x) for x in x.split()))
    
    return df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), tokenizer=LightPreProcessing, min_df=3, max_df=0.9, strip_accents='unicode', use_idf=True, smooth_idf=True, sublinear_tf=True)
vectorizer.fit(pd.concat([df_train['question_text'], df_test['question_text']]))


X = vectorizer.transform(df_train['question_text'])
y = df_train['target'].values

X_test = vectorizer.transform(df_test['question_text'])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
X

<1000000x1727882 sparse matrix of type '<class 'numpy.float64'>'
	with 33335948 stored elements in Compressed Sparse Row format>

## NB Features

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import csr_matrix
def createGaussian(MAT,alpha=1):
  a = alpha + X[y==1].sum(0)
  b = alpha + X[y==0].sum(0)
  convertMat = csr_matrix(np.log( (a/(alpha + (y==1).sum())) / (b/(alpha + (y==0).sum())) ) )
  return MAT.multiply(convertMat)

In [ ]:
X_nb = createGaussian(X,1)
X_test_nb = createGaussian(X_test,1)

In [ ]:
X_nb

<1000000x1727882 sparse matrix of type '<class 'numpy.float64'>'
	with 33335948 stored elements in Compressed Sparse Row format>

## Cross Validation & Testing

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
xTrainM = np.zeros(X_nb.shape[0])
xTestM = np.zeros(X_test_nb.shape[0])

listModels = []
sf = list(StratifiedKFold(n_splits=20, shuffle=True, random_state=42).split(df_train, y))
n = len(list(sf))
for i , (train_index, test_index) in enumerate(list(sf)):
    print("i")
    xTrainFold = X_nb[train_index]
    yTrainFold = y[train_index]
    xValFold = X_nb[test_index]
    yValFold = y[test_index]

    lr = LogisticRegression(solver='lbfgs', dual=False, class_weight='balanced', C=0.5, max_iter=500)
    lr.fit(xTrainFold,yTrainFold)
    listModels.append(lr)
    xTrainM[test_index] = (lr.predict_proba(X_nb[test_index]))[:,1]
    xTestM += lr.predict_proba(X_test_nb)[:,1]/n


i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i
i


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score
f1 = []
for i in range(100):
  f1.append(f1_score(y_true = y , y_pred = xTrainM > (i*0.01)))
t = f1.index(max(f1))


In [ ]:
threshold = t*0.01

In [ ]:
print(threshold)

0.67


In [ ]:
l = [ i for i in xTrainM if i > threshold ]

In [ ]:
len(l)

71491

In [ ]:
df_test

,qid,question_text
0,a4f3da3a3df9dd881edd,My period is due on my wedding day. How can I ...
1,9914c62ed3f69684d549,How many numbers higher than a million can be ...
2,8138ae48649e37091a91,"How come I feel nothing for my family, but sti..."
3,981b4753d17ef14d09f7,"In case of collapse of the Democratic party, w..."
4,452e2c705276ba16b7b7,Who is Émile Naoumoff?
...,...,...
306117,a352dff4fcc2571815ce,Did anyone get an update on Maruti Suzuki All ...
306118,ad4a8498d97c536c67b9,What 5 people in history do you find the most ...
306119,19784a27b55d4b453fda,How can I remove the tan on my forehead?
306120,370191dba26465997879,"If you are a well known hacker, will you be mo..."


In [ ]:
y.shape

(1000000,)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(X[0:,:], y[0:])
predicted = MNB.predict(X_test)

In [ ]:
l = [ i for i in predicted if i > 0 ]

In [ ]:
len(l)

2252

In [ ]:
t = (xTestM > threshold).astype(int)
l = [ i for i in range(len(predicted))if t[i] != predicted[i] ]

In [ ]:
ans = []
for i in range(len(predicted)) :
  ans.append(predicted[i] or t[i])

In [ ]:
ans

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
df_test.drop(columns=['question_text'], inplace=True)
df_test['target'] = (xTestM > threshold).astype(int)
# df_test['target'] = ans
df_test.to_csv('LR-NewNEWFINALThresh.csv', index=False)